In [1]:
import json
import os
import re
import string
from collections import defaultdict
from cltk.tokenize.word import WordTokenizer
from cltk.stem.lemma import LemmaReplacer
from cltk.stem.latin.j_v import JVReplacer
from nltk.translate import AlignedSent, Alignment,IBMModel1,IBMModel2,IBMModel3,PhraseTable
from nltk.translate.phrase_based import phrase_extraction,extract
from nltk.tokenize import wordpunct_tokenize
from nltk.translate.ibm_model import AlignmentInfo


In [2]:
lat_tokenizer = WordTokenizer('latin').tokenize
j = JVReplacer()
eng_tokenizer = wordpunct_tokenize
lem = LemmaReplacer('latin')

In [3]:
def tokenize_sentences(sent,tok):
    tokens = []
    if type(sent) == list:
        for s in sent:
            s = re.sub(r'[{}]'.format(string.punctuation),'',s).lower()
            s = re.sub(r'  *', ' ', s)
            tokens += tok(s.lower())
    else:
        sent = re.sub(r'[{}]'.format(string.punctuation),'',sent).lower()
        sent = re.sub(r'  *', ' ', sent)
        tokens += tok(sent)
        
    return tokens

In [4]:
os.chdir('../aligned_sentences/')

In [5]:
os.listdir()

['caes_bc_sentences.json',
 'cicero_lael_friendship_sentences.json',
 'vitruvius_sentences.json',
 'lacus_no_error_sentences.json',
 'catullus.json',
 'suetonius_sentences.json',
 'cicero_div_sentences.json',
 'celsus_sentences.json',
 'amm_lat_sentences.json',
 'cicero_off_sentences.json',
 'caes_bg_sentences_03457.json',
 'cicero_sen_falc_sentences.json',
 'lacus_fixed_error_sentences.json']

In [6]:
bitext = []

for f in os.listdir():
    if f == 'amm_lat_sentences.json':
        continue
    of = open(f)
    aligned_sentences = json.loads(of.read())
    of.close()

    for a in aligned_sentences:
        for s in a:
            lat_sent = [j.replace(t) for t in tokenize_sentences(s[0],lat_tokenizer)]
            eng_sent = tokenize_sentences(s[1], eng_tokenizer)
            if len(lat_sent) == 0 or len(eng_sent) == 0:
                continue
            bitext.append(AlignedSent(lat_sent,eng_sent))

of = open('amm_lat_sentences.json')
aligned_sentences = json.loads(of.read())
of.close()

for a in aligned_sentences:
    for p in a:
        for s in p:
            lat_sent = [j.replace(t) for t in tokenize_sentences(s[0],lat_tokenizer)]
            #while '' in lat_sent:
            #    lat_sent.remove('')
            #lat_sent = lem.lemmatize(lat_sent)
            eng_sent = tokenize_sentences(s[1], eng_tokenizer)
            if len(lat_sent) == 0 or len(eng_sent) == 0:
                continue
            bitext.append(AlignedSent(lat_sent,eng_sent))

In [7]:
len(bitext)

21953

In [ ]:
bitext[3].words

In [8]:
eng_bitext = []

for f in os.listdir():
    if f == 'amm_lat_sentences.json':
        continue
    of = open(f)
    aligned_sentences = json.loads(of.read())
    of.close()

    for a in aligned_sentences:
        for s in a:
            lat_sent = [j.replace(t) for t in tokenize_sentences(s[0],lat_tokenizer)]
            eng_sent = tokenize_sentences(s[1], eng_tokenizer)
            if len(lat_sent) == 0 or len(eng_sent) == 0:
                continue
            eng_bitext.append(AlignedSent(eng_sent, lat_sent))

of = open('amm_lat_sentences.json')
aligned_sentences = json.loads(of.read())
of.close()

for a in aligned_sentences:
    for p in a:
        for s in p:
            lat_sent = [j.replace(t) for t in tokenize_sentences(s[0],lat_tokenizer)]
            #while '' in lat_sent:
            #    lat_sent.remove('')
            #lat_sent = lem.lemmatize(lat_sent)
            eng_sent = tokenize_sentences(s[1], eng_tokenizer)
            if len(lat_sent) == 0 or len(eng_sent) == 0:
                continue
            eng_bitext.append(AlignedSent(eng_sent,lat_sent))

In [9]:
lat_mod = IBMModel2(bitext,6)

In [10]:
eng_mod = IBMModel2(eng_bitext,6)

In [11]:
print('done')

done


In [48]:
lat_mod.best_model2_alignment(AlignedSent(word,mots))

In [12]:
ebitext = eng_bitext

In [13]:
lbitext = bitext

In [44]:
word = lbitext[2].words
mots = lbitext[2].mots

In [49]:
a = _

In [53]:
a.alignment

(0, 2, 1, 2, 9, 8, 27, 10, 7, 40, 14, 20, 3, 22, 40, 40, 29, 16)

In [33]:
a.trg_sentence, a.src_sentence = a.src_sentence, a.trg_sentence

In [50]:
lat_mod.prob_t_a_given_s(alignment_info=a)

1e-12

In [54]:
for b in range(len(ebitext)):
    if ebitext[b].mots[-1] != 'NULL':
        ebitext[b].mots.append('NULL')
    if ebitext[b].words[-1] != 'NULL':
        ebitext[b].words.append('NULL')
    lens = [len(ebitext[b].words),len(ebitext[b].mots)]

    alin = [list(tup) for tup in ebitext[b].alignment]

    for i,a in enumerate(alin):
        if None in a:
            alin[i][alin[i].index(None)] = lens[a.index(None)]-1

    alin = [tuple(tup) for tup in alin]

    ebitext[b].alignment = Alignment(alin)

In [55]:
for b in range(len(lbitext)):
    if lbitext[b].mots[-1] != 'NULL':
        lbitext[b].mots.append('NULL')
    if lbitext[b].words[-1] != 'NULL':
        lbitext[b].words.append('NULL')
    lens = [len(lbitext[b].words),len(lbitext[b].mots)]

    alin = [list(tup) for tup in lbitext[b].alignment]

    for i,a in enumerate(alin):
        if None in a:
            alin[i][alin[i].index(None)] = lens[a.index(None)]-1

    alin = [tuple(tup) for tup in alin]

    lbitext[b].alignment = Alignment(alin)

In [56]:
ephrases = defaultdict(lambda: [0,defaultdict(int)])
skip = [1349,9230]
base = .5
for b in range(len(ebitext)): 
    if b in skip:
        continue
    if b == base*1000:
        print(b)
        base += .5
    
    pe = phrase_extraction(' '.join(ebitext[b].words), ' '.join(ebitext[b].mots), ebitext[b].alignment,max_phrase_length=4)
    pe = list(pe)
    for phrase in pe:
        english_tokens = eng_tokenizer(phrase[2])
        lat_tokens = lat_tokenizer(phrase[3])
        
        if len(english_tokens) < 5 and len(lat_tokens) < 5:
            ephrases[tuple(english_tokens)][1][tuple(lat_tokens)] += 1
            ephrases[tuple(english_tokens)][0] += 1

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500


In [58]:
b

7997

In [59]:
skip = [7997]

lphrases = defaultdict(lambda: [0,defaultdict(int)])
base = .5
for b in range(len(lbitext)): 
    if b in skip:
        continue
    if b == base*1000:
        print(b)
        base += .5
    pe = phrase_extraction(' '.join(lbitext[b].words), ' '.join(lbitext[b].mots), lbitext[b].alignment,max_phrase_length=4)    
    pe = list(pe)
    for phrase in pe:
        english_tokens = eng_tokenizer(phrase[3])
        lat_tokens = lat_tokenizer(phrase[2])
        
        if len(english_tokens) < 5 and len(lat_tokens) < 5:
            lphrases[tuple(lat_tokens)][1][tuple(english_tokens)] += 1
            lphrases[tuple(lat_tokens)][0] += 1

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500


In [62]:
len(ephrases)

29866

In [151]:
phrases = defaultdict(lambda: [0,defaultdict(int)])
untranslated = dict()

for phrase in list(ephrases.keys()):
    all_added = True
    for trg_phrase in ephrases[phrase][1]:
        if lphrases[trg_phrase][0] != 0:
            for sub_phrase in lphrases[trg_phrase][1]:
                if sub_phrase == phrase:
                    phrases[phrase][1][trg_phrase] += ephrases[phrase][1][trg_phrase]
                    phrases[phrase][0] += ephrases[phrase][1][trg_phrase]
                else:
                    all_added = False
    if not all_added:
        untranslated[phrase] = ephrases[phrase]

In [152]:
for k in untranslated:  # while there are still untranslated phrases
    maximum = 0
    for possible_translation in untranslated[k][1]:
        if untranslated[k][1][possible_translation] > maximum:
            maximum = untranslated[k][1][possible_translation]
            translation = possible_translation
    phrases[k][0] = maximum
    phrases[k][1][translation] = maximum

In [153]:
phrase_table = PhraseTable()

In [154]:
from numpy import log

In [155]:
for k in phrases:
    for t in phrases[k][1]:
        phrase_table.add(k,t,log(phrases[k][1][t]/phrases[k][0]))

In [68]:
import dill as pickle

In [69]:
with open('../pkls/latin_lang_model.pk', 'rb') as fin:
    lang_model = pickle.load(fin)

In [70]:
from nltk.translate.stack_decoder import StackDecoder

In [263]:
def language_prob(phrase):
    phrase = list(phrase)
    if phrase == ['NULL']:
        return 1

    while 'NULL' in phrase and phrase != []:
        phrase.remove('NULL')

    if phrase == []:
        return 1
    product = 1
    for i in range(len(phrase)-2):
        product *= lang_model.S(phrase[i+2],(phrase[i],phrase[i+1]))
    return log(product)

In [266]:
language_model = type('',(object,),{'probability_change': lambda self, context, phrase: language_prob(phrase), 'probability': lambda self, phrase: language_prob(phrase)})()
SD = StackDecoder(phrase_table=phrase_table,language_model=language_model,)

In [571]:
with open('../pkls/translation_model_1.pk', 'wb') as fout:
    pickle.dump(SD,fout)

In [554]:
SD.translate(['the','men','give','flowers','to','their','wives'])

['NULL', 'homines', 'dare', 'flores', 'ad', 'coniugibus']

In [555]:
SD.translate(['the','dog','is','here'])

['NULL', 'canis', 'est', 'hic']

In [556]:
SD.translate(['the','road','is','blocked'])

['NULL', 'uia', 'est', 'obstructis']

In [557]:
SD.translate(['caesar','is','alive'])

['caesar', 'est', 'uiuus']

In [558]:
SD.translate(['there','is','no','way'])

['nulla', 'est', 'uia']

In [559]:
SD.translate(['he','himself','thought','so'])

['ipse', 'existimauit', 'ita']

In [560]:
SD.translate(['he','himself','says','so'])

['ipse', 'ait', 'ita']

In [546]:
SD.translate(['i','have','said','so'])

/home/ubuntu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:14: RuntimeWarning: divide by zero encountered in log


['dixi', 'ita']

In [545]:
SD.translate(['the','man','has','weapons'])

['NULL', 'uir', 'habet', 'tela']

In [561]:
SD.translate(['the','road','is','dangerous'])

['NULL', 'uia', 'periculosum', 'est']

In [562]:
SD.translate(['who','are','they'])

['qui', 'sunt', 'eos']

In [563]:
SD.translate(['who','are','you'])

['qui', 'es']

In [564]:
SD.translate(['where','are','you'])

['ubi', 'es']

In [565]:
SD.translate(['where','is','it'])

['ubi', 'est']

In [566]:
SD.translate(['there','were','three','men','here'])

['ibi', 'erant', 'tres', 'homines', 'hic']

In [567]:
SD.translate(['there','is','an','enemy','in','our','city'])

['est', 'hostis', 'in', 'nostris', 'urbem']

In [544]:
SD.translate(['an','enemy','in','our','city'])

['hostis', 'in', 'nostris', 'urbem']

In [568]:
SD.translate(['an','enemy','at','the','gate'])

['hostis', 'apud', 'NULL', 'portam']

In [569]:
SD.translate(['an','enemy'])

['hostis']

In [570]:
SD.translate(['our','house','is','far'])

['nostri', 'domum', 'est', 'longe']

In [396]:
SD.translate(['the','last','time'])

['NULL', 'ultimum', 'tempus']

In [380]:
SD.translate(['the','person','is','not','good'])

['NULL', 'persona', 'non', 'est', 'boni']

In [381]:
SD.translate(['to','a','friend'])

['ad', 'amicum']

In [382]:
SD.translate(['i','think','so'])

/home/ubuntu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:14: RuntimeWarning: divide by zero encountered in log


['puto', 'ita']

In [383]:
SD.translate(['are','you','sure'])

['es', 'certa']

In [384]:
SD.translate(['i','am','not','unhappy'])

/home/ubuntu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:14: RuntimeWarning: divide by zero encountered in log


['sum', 'non', 'miser']

In [385]:
SD.translate(['you','are','a','great','person'])

['es', 'magna', 'persona']

In [ ]:
SD.stack_size

In [259]:
SD.distortion_factor = 1

In [543]:
phrase_table.translations_for(('love',))

[PhraseTableEntry(trg_phrase=('amor',), log_prob=0.0),
 PhraseTableEntry(trg_phrase=('amorem',), log_prob=-0.095310179804324893),
 PhraseTableEntry(trg_phrase=('amore',), log_prob=-0.20067069546215111),
 PhraseTableEntry(trg_phrase=('amare',), log_prob=-0.31845373111853459),
 PhraseTableEntry(trg_phrase=('amoris',), log_prob=-1.2992829841302609),
 PhraseTableEntry(trg_phrase=('diligere',), log_prob=-1.2992829841302609),
 PhraseTableEntry(trg_phrase=('amorem', 'NULL'), log_prob=-2.3978952727983707)]